<a href="https://www.kaggle.com/code/shivpalyadav/spaceship-titanic-by-shiv?scriptVersionId=167195751" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Remove warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Data wrangling
import pandas as pd
import numpy as np
import missingno
from collections import Counter


In [ ]:
# Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Machine learning models
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

In [ ]:
# Model evaluation
from sklearn.model_selection import cross_val_score

In [ ]:
# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

In [ ]:
train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
ss = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
ss.head()

In [ ]:
train.info()
print('-'*45)
test.info()

In [ ]:
# Missing data in training set by columns

train.isnull().sum().sort_values(ascending = False)

In [ ]:
# Missing data in test set by columns

test.isnull().sum().sort_values(ascending = False)

In [ ]:
# Missing data in training set

missingno.matrix(train)


In [ ]:
# Missing data in test set

missingno.matrix(test)

In [ ]:
# Summary statistics for training set 

train.describe()

In [ ]:
test.shape

In [ ]:
train.shape

In [ ]:
train.describe

In [ ]:
train.describe

In [ ]:
#Age distribution
sns.histplot(test['Age'].dropna(), bins=20, kde=True)
plt.title('Age Distribution')
plt.show()

In [ ]:
#Age distribution
sns.histplot(train['Age'].dropna(), bins=20, kde=True)
plt.title('Age Distribution')
plt.show()

In [ ]:
train['Age'].isnull().sum()

In [ ]:
test['Age'].isnull().sum()

In [ ]:
# Age distribution
plt.hist(test['Age'], bins=20, color=('skyblue'), edgecolor='black')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Age distribution
plt.hist(train['Age'], bins=20, color=('skyblue'), edgecolor='black')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
categorical_columns = train.select_dtypes(include=['object']).columns
numerical_columns = train.select_dtypes(include=[np.number]).columns

In [ ]:
for col in categorical_columns:
    print(f"Unique values in '{col}':")
    print(train[col].unique())
    print("\n")

In [ ]:
for col in numerical_columns:
    unique_values = train[col].nunique()
    print(f"Unique values in '{col}': {unique_values}")


In [ ]:
num_duplicate_rows_train = train.duplicated().sum()
print("Number of duplicate values in train DataFrame:", num_duplicate_rows_train)

In [ ]:
num_duplicate_rows_test = test.duplicated().sum()
print("Number of duplicate values in test DataFrame:", num_duplicate_rows_test)

In [ ]:
missingValuesTrain = train.isnull().sum()
print("Number of missing values in each column of train DataFrame:")
print(missingValuesTrain)

In [ ]:
missingValuesTest = test.isnull().sum()
print("Number of missing values in each column of test DataFrame:")
print(missingValuesTest)